### More `nographs` stuff: Iteration over outgoing edges, instead of nodes

In [ ]:
"""Iteration over outgoing edges, instead of nodes, see
https://nographs.readthedocs.io/en/latest/
graphs_and_adaptation.html#graphs-with-attributes"""
from functools import wraps
from decorator import decorator
import nographs as nog
from graphviz import Digraph

In [ ]:
node_style ={"style": 'filled',            "shape": 'box',
             "align": 'left',              "fontsize": '12',
             "ranksep":'0.01',             "height":'0.02',   "width":'0.04',}
gr_style = {"linelength": '16', "rankdir": 'LR'}#"landscape":'True',

def adapter_graphviz( wrapped ):
    """translation to GraphViz visitor"""
    dot = Digraph( node_attr = node_style, graph_attr = gr_style, )
    @wraps( wrapped )
    def wrapper_bg( _edge, _trav ):
        dot.node( str(_edge), str(_edge) )
        for result in wrapped(_edge, _trav):
            dot.edge( str(_edge), str(result[0]), str(result[1]), )
            yield result
    wrapper_bg.dot = dot
    return wrapper_bg

Optional feature, uncomment `@filter_big` to apply

In [ ]:
@decorator
def filter_big( wrapped, _edge, _trav ):
    """Decorator for filtering, lets shrink graph a bit"""
    yield from filter( lambda e:e[0]<50, wrapped( _edge, _trav ) )

In [ ]:
@adapter_graphviz
#@filter_big    #uncomment to shrink the graph (search space)
def next_edges(i, _):
    """From graphs_and_adaptation.html#graphs-with-attributes"""
    yield i + 1, "i+1"
    yield 2 * i, "2*i"
    yield i * i, "i*i"

traversal = nog.TraversalBreadthFirst(next_labeled_edges=next_edges)
_ = traversal.start_from(0, build_paths=True)

for edge in traversal.paths[ traversal.go_to(49) ]:
    print(edge)

next_edges.dot              # pylint: disable=pointless-statement

In [ ]:
next_edges.dot.attr( layout="neato")
next_edges.dot              # pylint: disable=pointless-statement